In [35]:
import os
import json

import torch
import spacy
import pyserini


ModuleNotFoundError: No module named 'spacy'

In [24]:
if not os.path.isdir("data"):
    os.mkdir("data")

if not os.path.isfile("data\\context_articles.json"):
    !curl -LO https://iirc-dataset.s3.us-west-2.amazonaws.com/context_articles.tar.gz
    !move context_articles.tar.gz data
    !tar -xf data/context_articles.tar.gz
    !move context_articles.json data

if not os.path.isfile("data\\iirc_test.json"):
    !curl -LO https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_test.json
    !move iirc_test.json data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  2 2807k    2 69213    0     0  74427      0  0:00:38 --:--:--  0:00:38 74502
 55 2807k   55 1546k    0     0   775k      0  0:00:03  0:00:01  0:00:02  776k
100 2807k  100 2807k    0     0  1193k      0  0:00:02  0:00:02 --:--:-- 1194k


        1 arquivo(s) movido(s).


In [7]:
file = open("data\\context_articles.json", "r")
articles = json.load(file)
file.close()

In [27]:
file = open("data\\iirc_test.json", "r")
test_data = json.load(file)
file.close()

In [25]:
n_question = 50

In [29]:
documents = []
all_titles = []

documents_not_found = []

for i in range(n_question):
    item = test_data[i]
    if item['title'].lower() not in all_titles:
        documents.append({
                "title": item['title'],
                "content": item["text"]
            }
        )
        all_titles.append(item['title'].lower())
    for link in item["links"]:
        if link['target'].lower() in articles and link['target'].lower() not in all_titles:
            documents.append({
                "title": link['target'],
                "content": articles[link['target'].lower()]
            })
            all_titles.append(link['target'].lower())
        else:
            documents_not_found.append(link["target"].lower())

In [33]:
len(documents_not_found); len(documents)

733

In [34]:
nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

#12345
#   45678
stride = 3#2
max_length = 5#3

def window(documents, stride, max_length):
    treated_documents = []

    for j,document in enumerate(tqdm.tqdm(documents)):
        doc_text = document['content']
        doc = nlp(doc_text[:10000])
        sentences = [sent.text.strip() for sent in doc.sents]
        for i in range(0, len(sentences), stride):
            segment = ' '.join(sentences[i:i + max_length])
            treated_documents.append({
                "title": document['title'],
                "contents": document['title']+". "+segment,
                "segment": segment
            })
            if i + max_length >= len(sentences):
                break
    return treated_documents

treated_documents = window(documents, stride, max_length)

NameError: name 'spacy' is not defined